In [1]:
# import sys

# !{sys.executable} -m pip install pyodbc pandas prettytable

In [2]:
import pyodbc
import pandas as pd
from prettytable import PrettyTable
import warnings

# Suppress the specific warning
warnings.filterwarnings('ignore', 
                       message='pandas only supports SQLAlchemy connectable.*')

def connect_to_access(db_path):
    """Establish connection to Access database"""
    driver = 'Microsoft Access Driver (*.mdb, *.accdb)'
    conn_str = f'DRIVER={{{driver}}};DBQ={db_path};'
    return pyodbc.connect(conn_str)

def get_table_schema(conn, table_name):
    """Get schema information for a specific table"""
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM [{table_name}] WHERE 1=0")  # Get metadata only
    columns = [column[0] for column in cursor.description]
    cursor.close()
    return columns


In [3]:
def compare_schemas(db1, db2):
    """Session 1: Compare table schemas between databases"""
    conn1 = None
    conn2 = None
    try:
        conn1 = connect_to_access(db1)
        conn2 = connect_to_access(db2)
        
        # Get all tables from both databases
        tables1 = [table.table_name for table in conn1.cursor().tables(tableType='TABLE')]
        tables2 = [table.table_name for table in conn2.cursor().tables(tableType='TABLE')]
        
        all_tables = set(tables1 + tables2)
        results = []
        
        for table in sorted(all_tables):
            in_db1 = table in tables1
            in_db2 = table in tables2
            
            if not in_db1:
                results.append([table, "Only in DB2", "", ""])
                continue
            if not in_db2:
                results.append([table, "Only in DB1", "", ""])
                continue
                
            # Compare schemas
            schema1 = get_table_schema(conn1, table)
            schema2 = get_table_schema(conn2, table)
            
            if schema1 == schema2:
                results.append([table, "Identical", len(schema1), ""])
            else:
                diff_cols = list(set(schema1) - set(schema2)) + list(set(schema2) - set(schema1))
                results.append([table, "Different", f"{len(schema1)} vs {len(schema2)}", ", ".join(diff_cols)])
        
        # Format results as table
        pt = PrettyTable()
        pt.title = "Table Schema Comparison"
        pt.field_names = ["Table Name", "Status", "Column Count", "Differences"]
        pt.align = "l"
        for row in results:
            pt.add_row(row)
        
        print(pt)
        
    except Exception as e:
        print(f"Error comparing schemas: {str(e)}")
        
    finally:
        for conn in [conn1, conn2]:
            if conn:
                try:
                    conn.close()
                except Exception as e:
                    print(f"Error closing connection: {str(e)}")

In [4]:
def compare_row_counts(db1, db2):
    """Session 2: Compare row counts between databases"""
    conn1 = None
    conn2 = None
    try:
        conn1 = connect_to_access(db1)
        conn2 = connect_to_access(db2)
        
        # Get common tables
        tables1 = [table.table_name for table in conn1.cursor().tables(tableType='TABLE')]
        tables2 = [table.table_name for table in conn2.cursor().tables(tableType='TABLE')]
        common_tables = sorted(set(tables1) & set(tables2))
        
        results = []
        
        for table in common_tables:
            try:
                count1 = pd.read_sql(f"SELECT COUNT(*) FROM [{table}]", conn1).iloc[0,0]
                count2 = pd.read_sql(f"SELECT COUNT(*) FROM [{table}]", conn2).iloc[0,0]
                
                status = "Equal" if count1 == count2 else f"Diff: {abs(count1 - count2)} rows"
                results.append([table, count1, count2, status])
            except Exception as e:
                results.append([table, "Error", "Error", f"Count failed: {str(e)}"])
        
        # Format results as table
        pt = PrettyTable()
        pt.title = "Row Count Comparison"
        pt.field_names = ["Table Name", "DB1 Count", "DB2 Count", "Status"]
        pt.align = "l"
        for row in results:
            pt.add_row(row)
        
        print(pt)
        
    except Exception as e:
        print(f"Error comparing row counts: {str(e)}")
        
    finally:
        for conn in [conn1, conn2]:
            if conn:
                try:
                    conn.close()
                except Exception as e:
                    print(f"Error closing connection: {str(e)}")

In [5]:
import sys
import os

def compare_all_table_data(db1, db2, rpt_file_name='data_comparison_report.txt'):
    """Session 3: Compare data row by row for ALL tables with proper connection cleanup"""
    conn1 = None
    conn2 = None
    try:
        # Establish connections
        conn1 = connect_to_access(db1)
        conn2 = connect_to_access(db2)
        
        # Get common tables
        tables1 = [table.table_name for table in conn1.cursor().tables(tableType='TABLE')]
        tables2 = [table.table_name for table in conn2.cursor().tables(tableType='TABLE')]
        common_tables = sorted(set(tables1) & set(tables2))
        
        # Create a report file
        with open(rpt_file_name, 'w') as report_file:
            report_file.write("=== Detailed Data Comparison Report ===\n\n")
            
            for table in common_tables:
                try:
                    # Get all data from both tables
                    df1 = pd.read_sql(f"SELECT * FROM [{table}]", conn1)
                    df2 = pd.read_sql(f"SELECT * FROM [{table}]", conn2)
                    
                    # Prepare comparison
                    report_file.write(f"\n\n===== Table: {table} =====\n")
                    
                    # Compare row counts first
                    count_diff = len(df1) - len(df2)
                    if count_diff != 0:
                        report_file.write(f"ROW COUNT DIFFERENCE: {abs(count_diff)} rows ({len(df1)} vs {len(df2)})\n")
                    
                    # Find all columns that exist in both tables
                    common_cols = list(set(df1.columns) & set(df2.columns))
                    
                    # Find natural key candidates
                    key_candidates = [col for col in common_cols if 'id' in col.lower() or 'code' in col.lower()]
                    if not key_candidates:
                        key_candidates = common_cols  # Fall back to all columns if no obvious keys
                    
                    # Initialize counters
                    only_in_primary = []
                    only_in_backup = []
                    modified_rows = []
                    matched_rows = []
                    
                    # Create dictionaries for faster lookup
                    dict1 = {tuple(row[k] for k in key_candidates): row for _, row in df1.iterrows()}
                    dict2 = {tuple(row[k] for k in key_candidates): row for _, row in df2.iterrows()}
                    
                    all_keys = set(dict1.keys()).union(set(dict2.keys()))
                    
                    for key in all_keys:
                        in_primary = key in dict1
                        in_backup = key in dict2
                        
                        if in_primary and in_backup:
                            row1 = dict1[key]
                            row2 = dict2[key]
                            
                            # Compare all common columns
                            is_different = any(str(row1[col]) != str(row2[col]) for col in common_cols)
                            
                            if is_different:
                                modified_rows.append((row1, row2))
                            else:
                                matched_rows.append(row1)
                        elif in_primary:
                            only_in_primary.append(dict1[key])
                        elif in_backup:
                            only_in_backup.append(dict2[key])
                    
                    # Generate report
                    report_file.write("\n=== Summary ===\n")
                    report_file.write(f"Matching rows: {len(matched_rows)}\n")
                    report_file.write(f"Rows only in primary DB: {len(only_in_primary)}\n")
                    report_file.write(f"Rows only in backup DB: {len(only_in_backup)}\n")
                    report_file.write(f"Rows with different values: {len(modified_rows)}\n")
                    
                    # Report modified rows with differences highlighted
                    if modified_rows:
                        report_file.write("\n\n--- Rows with different values ---\n")
                        for row1, row2 in modified_rows:
                            report_file.write(f"\nKey: {', '.join(f'{k}={row1[k]}' for k in key_candidates)}\n")
                            
                            diff_cols = [col for col in common_cols 
                                        if str(row1[col]) != str(row2[col])]
                            
                            for col in diff_cols:
                                report_file.write(f"{col}:\n")
                                report_file.write(f"  Primary: {row1[col]}\n")
                                report_file.write(f"  Backup:  {row2[col]}\n")
                                report_file.write("---\n")
                    
                    # Report rows only in primary DB
                    if only_in_primary:
                        report_file.write("\n--- Rows only in PRIMARY database ---\n")
                        for row in only_in_primary:
                            report_file.write(row.to_string() + "\n\n")
                    
                    # Report rows only in backup DB
                    if only_in_backup:
                        report_file.write("\n--- Rows only in BACKUP database ---\n")
                        for row in only_in_backup:
                            report_file.write(row.to_string() + "\n\n")
                
                except Exception as table_err:
                    report_file.write(f"\nERROR processing table {table}: {str(table_err)}\n")
                    continue
        
    except Exception as main_err:
        print(f"Fatal error during comparison: {str(main_err)}")
        if 'report_file' in locals():
            report_file.write(f"\nFATAL ERROR: {str(main_err)}\n")
    
    finally:
        # Ensure connections are properly closed
        for conn in [conn1, conn2]:
            if conn:
                try:
                    # Commit any pending transactions
                    try:
                        conn.commit()
                    except:
                        pass
                    
                    # Close the connection (pyodbc doesn't have cursors attribute)
                    conn.close()
                    
                    # Additional cleanup for Access lock files
                    if sys.platform == 'win32':
                        db_path = db1 if conn == conn1 else db2
                        lock_file = os.path.splitext(db_path)[0] + '.laccdb'
                        if os.path.exists(lock_file):
                            try:
                                os.remove(lock_file)
                            except:
                                pass
                
                except Exception as close_err:
                    print(f"Error closing connection: {str(close_err)}")
        
        print(f"\nDetailed data comparison report saved to '{rpt_file_name}'")

In [7]:
db1_path = r'C:\tmp\access_to_sqlite\Database1_be.accdb'  # Update path
db2_path = r'C:\tmp\access_to_sqlite\Database1_be_bak.accdb'  # Update path

print("\n=== Session 1: Table Schema Comparison ===")
compare_schemas(db1_path, db2_path)

print("\n=== Session 2: Row Count Comparison ===")
compare_row_counts(db1_path, db2_path)

print("\n=== Session 3: Row-by-Row Data Comparison ===")
rpt_file_name = 'data_comparison_report.txt'
compare_all_table_data(db1_path, db2_path, rpt_file_name=rpt_file_name)

with open(rpt_file_name, 'r') as f:
    print(f.read())


=== Session 1: Table Schema Comparison ===
+------------------------------------------------------------+
|                  Table Schema Comparison                   |
+-------------------+-----------+--------------+-------------+
| Table Name        | Status    | Column Count | Differences |
+-------------------+-----------+--------------+-------------+
| Customers         | Identical | 17           |             |
| Employees         | Identical | 17           |             |
| NorthwindFeatures | Identical | 6            |             |
| OrderDetails      | Identical | 9            |             |
| OrderStatus       | Identical | 8            |             |
| Orders            | Identical | 12           |             |
| Products          | Identical | 9            |             |
| SystemSettings    | Identical | 4            |             |
| Welcome           | Identical | 5            |             |
+-------------------+-----------+--------------+-------------+

=== Sessio